In [1]:
import sys
sys.path.append("../classes")

In [2]:
import pickle
from sklearn.svm import SVC

from sklearn.feature_selection import SelectKBest
from sklearn.decomposition import PCA

import numpy as np, pandas as pd
from geno_classifier import *
from geno_utils import parse_database
from itertools import starmap

import GEOparse

("Neoplasms by site"[MeSH Terms]) AND normal[Subset Description] 

# uterine smooth muscle tumors

In [3]:
uterine = GEOparse.get_GEO(filepath='../GDS533_full.soft.gz').table

25-Apr-2018 09:58:58 INFO GEOparse - Parsing ../GDS533_full.soft.gz: 
25-Apr-2018 09:58:58 DEBUG GEOparse - DATABASE: Geo
25-Apr-2018 09:58:58 DEBUG GEOparse - DATASET: GDS533
25-Apr-2018 09:58:58 DEBUG GEOparse - SUBSET: GDS533_1
25-Apr-2018 09:58:58 DEBUG GEOparse - SUBSET: GDS533_2
25-Apr-2018 09:58:58 DEBUG GEOparse - SUBSET: GDS533_3
25-Apr-2018 09:58:59 DEBUG GEOparse - SUBSET: GDS533_4
25-Apr-2018 09:58:59 DEBUG GEOparse - ANNOTATION: 
25-Apr-2018 09:58:59 ERROR GEOparse - Cannot recognize type Annotation
25-Apr-2018 09:58:59 DEBUG GEOparse - DATASET: GDS533
/home/abdu/Desktop/genobolitics/venv/lib/python3.5/site-packages/GEOparse/GEOparse.py:332 FutureWarning: from_csv is deprecated. Please use read_csv(...) instead. Note that some of the default arguments are different, so please refer to the documentation for from_csv when changing your function calls


In [4]:
healthy_raw = """
#GSM11625 = Value for GSM11625: Myo43; src: myomtrium
#GSM11757 = Value for GSM11757: Myo243; src: myomtrium
#GSM11758 = Value for GSM11758: Myo44; src: myomtrium
#GSM11759 = Value for GSM11759: Myo220; src: myomtrium
#GSM11761 = Value for GSM11761: Leio216; src: uterine leiomyoma
#GSM11762 = Value for GSM11762: Leio217; src: uterine leiomyoma
#GSM11763 = Value for GSM11763: Leio218; src: uterine leiomyoma
#GSM11764 = Value for GSM11764: Leio219; src: uterine leiomyoma
#GSM11765 = Value for GSM11765: Leio240; src: uterine leiomyoma
#GSM11766 = Value for GSM11766: Leio241; src: uterine leiomyoma
#GSM11767 = Value for GSM11767: Leio242; src: uterine leiomyoma"""

In [5]:
unhealthy_raw = """
#GSM11760 = Value for GSM11760: LMS19825; src: uterine leiomyosarcoma
#GSM11768 = Value for GSM11768: LMS278; src: uterine leiomyosarcoma
#GSM11769 = Value for GSM11769: LMS818; src: uterine leiomyosarcoma
#GSM11770 = Value for GSM11770: LMS906; src: uterine leiomyosarcoma
#GSM11771 = Value for GSM11771: LMS1775; src: uterine leiomyosarcoma
#GSM11772 = Value for GSM11772: LMS2103; src: uterine leiomyosarcoma
#GSM11773 = Value for GSM11773: LMS4578; src: uterine leiomyosarcoma
#GSM11774 = Value for GSM11774: LMS11987; src: uterine leiomyosarcoma
#GSM11775 = Value for GSM11775: LMS19597; src: uterine leiomyosarcoma
#GSM11776 = Value for GSM11776: exULMS2906; src: extra-uterine leiomyosarcoma
#GSM11777 = Value for GSM11777: exULMS91; src: extra-uterine leiomyosarcoma
#GSM11778 = Value for GSM11778: exULMS92; src: extra-uterine leiomyosarcoma
#GSM11779 = Value for GSM11779: exULMS781; src: extra-uterine leiomyosarcoma"""

In [6]:
clean = lambda string: string.replace(':', '').split()
predicate = lambda word: word.startswith('GSM')

healthy1 = set(filter(predicate, clean(healthy_raw)))
unhealthy1 = set(filter(predicate, clean(unhealthy_raw)))

In [7]:
len(healthy1),len(unhealthy1)

(11, 13)

# tongue cancer

In [8]:
tongue = GEOparse.get_GEO(filepath='../GDS4562_full.soft.gz').table

25-Apr-2018 09:58:59 INFO GEOparse - Parsing ../GDS4562_full.soft.gz: 
25-Apr-2018 09:58:59 DEBUG GEOparse - DATABASE: Geo
25-Apr-2018 09:58:59 DEBUG GEOparse - DATASET: GDS4562
25-Apr-2018 09:58:59 DEBUG GEOparse - SUBSET: GDS4562_1
25-Apr-2018 09:58:59 DEBUG GEOparse - SUBSET: GDS4562_2
25-Apr-2018 09:58:59 DEBUG GEOparse - SUBSET: GDS4562_3
25-Apr-2018 09:58:59 DEBUG GEOparse - SUBSET: GDS4562_4
25-Apr-2018 09:58:59 DEBUG GEOparse - SUBSET: GDS4562_5
25-Apr-2018 09:58:59 DEBUG GEOparse - SUBSET: GDS4562_6
25-Apr-2018 09:58:59 DEBUG GEOparse - SUBSET: GDS4562_7
25-Apr-2018 09:58:59 DEBUG GEOparse - SUBSET: GDS4562_8
25-Apr-2018 09:58:59 DEBUG GEOparse - SUBSET: GDS4562_9
25-Apr-2018 09:58:59 DEBUG GEOparse - SUBSET: GDS4562_10
25-Apr-2018 09:58:59 DEBUG GEOparse - SUBSET: GDS4562_11
25-Apr-2018 09:58:59 DEBUG GEOparse - SUBSET: GDS4562_12
25-Apr-2018 09:58:59 DEBUG GEOparse - SUBSET: GDS4562_13
25-Apr-2018 09:58:59 DEBUG GEOparse - SUBSET: GDS4562_14
25-Apr-2018 09:58:59 DEBUG GEOpar

In [9]:
healthy_raw2 = """
#GSM771246 = Value for GSM771246: HN07.356N; src: Normal
#GSM771169 = Value for GSM771169: HN08.383N; src: Normal
#GSM771172 = Value for GSM771172: HN08.424N; src: Normal
#GSM771176 = Value for GSM771176: HN08.429N; src: Normal
#GSM771179 = Value for GSM771179: HN08.433; src: Normal
#GSM771182 = Value for GSM771182: HN04.33N; src: Normal
#GSM771186 = Value for GSM771186: HN04.93N; src: Normal
#GSM771189 = Value for GSM771189: HN05.180N; src: Normal
#GSM771200 = Value for GSM771200: HN05.213N; src: Normal
#GSM771209 = Value for GSM771209: HN02.162N; src: Normal
#GSM771212 = Value for GSM771212: HN02.174N; src: Normal
#GSM771214 = Value for GSM771214: HN04.65N; src: Normal
#GSM771222 = Value for GSM771222: HN04.84N; src: Normal
#GSM771225 = Value for GSM771225: HN04.104N; src: Normal
#GSM771232 = Value for GSM771232: HN04.172N; src: Normal
#GSM771239 = Value for GSM771239: HN04.19N; src: Normal tongue
#GSM771195 = Value for GSM771195: HN05.191N; src: Normal
#GSM771205 = Value for GSM771205: HN02.135N; src: Normal
#GSM771227 = Value for GSM771227: HN04.137N; src: Normal
#GSM771236 = Value for GSM771236: HN07.342N; src: Normal
#GSM771152 = Value for GSM771152: HNO2-19916N; src: Normal
#GSM771155 = Value for GSM771155: HNO4-87N; src: Normal
#GSM771159 = Value for GSM771159: HNO4-104N; src: Normal
#GSM771163 = Value for GSM771163: HNO5-137N; src: Normal
"""

In [10]:
unhealthy_raw2 = """
#GSM771244 = Value for GSM771244: HN07.354T; src: FOM tumor
#GSM771170 = Value for GSM771170: HN08.384T; src: Tumor
#GSM771173 = Value for GSM771173: HN08.425T; src: Tumor
#GSM771177 = Value for GSM771177: HN08.430T; src: Tumor
#GSM771180 = Value for GSM771180: HN08.434; src: Tumor
#GSM771183 = Value for GSM771183: HN04.34T; src: Tumor
#GSM771187 = Value for GSM771187: HN04.94T; src: Tumor
#GSM771190 = Value for GSM771190: HN05.181T; src: Tumor
#GSM771201 = Value for GSM771201: HN05.214T; src: Tumor
#GSM771210 = Value for GSM771210: HN02.171T; src: Tumor
#GSM771215 = Value for GSM771215: HN04.66T; src: Tumor
#GSM771221 = Value for GSM771221: HN04.83T; src: Tumor
#GSM771224 = Value for GSM771224: HN04.103T; src: Tumor
#GSM771233 = Value for GSM771233: HN04.173T; src: Tumor
#GSM771243 = Value for GSM771243: HN04.24T; src: Tongue tumor
#GSM771196 = Value for GSM771196: HN05.192T; src: Tumor
#GSM771203 = Value for GSM771203: HN02.129T; src: Tumor
#GSM771228 = Value for GSM771228: HN04.138T; src: Tumor
#GSM771237 = Value for GSM771237: HN07.343T; src: Tumor
#GSM771153 = Value for GSM771153: HNO2-19916T; src: Tumor
#GSM771158 = Value for GSM771158: HNO4-86T; src: Tumor
#GSM771162 = Value for GSM771162: HNO4-107T; src: Tumor
#GSM771164 = Value for GSM771164: HNO5-138T; src: Tumor
"""

In [11]:
clean = lambda string: string.replace(':', '').split()
predicate = lambda word: word.startswith('GSM')

healthy2 = set(filter(predicate, clean(healthy_raw2)))
unhealthy2 = set(filter(predicate, clean(unhealthy_raw2)))

In [12]:
print(len(healthy2), len(unhealthy2))

24 23


In [13]:
labels1 = [(h, 'healthy') for h in healthy1] + [(d, 'unhealthy') for d in unhealthy1]
labels2 = [(h, 'healthy') for h in healthy2] + [(d, 'unhealthy') for d in unhealthy2]

In [14]:
X1, y1 = parse_database(uterine, labels=dict(labels1), n_jobs=-1)
X2, y2 = parse_database(tongue, labels=dict(labels2), n_jobs=-1)

[Parallel(n_jobs=-1)]: Done   2 tasks      | elapsed:  4.1min
[Parallel(n_jobs=-1)]: Done   9 tasks      | elapsed:  4.1min
[Parallel(n_jobs=-1)]: Done  12 out of  24 | elapsed:  4.2min remaining:  4.2min
[Parallel(n_jobs=-1)]: Done  15 out of  24 | elapsed:  4.2min remaining:  2.5min
[Parallel(n_jobs=-1)]: Done  18 out of  24 | elapsed:  4.2min remaining:  1.4min
[Parallel(n_jobs=-1)]: Done  21 out of  24 | elapsed:  4.2min remaining:   36.3s
[Parallel(n_jobs=-1)]: Done  24 out of  24 | elapsed:  4.3min remaining:    0.0s
[Parallel(n_jobs=-1)]: Done  24 out of  24 | elapsed:  4.3min finished


GSM11767 added with length 5431
GSM11762 added with length 5431
GSM11761 added with length 5431
GSM11772 added with length 5431
GSM11757 added with length 5431
GSM11777 added with length 5431
GSM11760 added with length 5431
GSM11625 added with length 5431
GSM11765 added with length 5431
GSM11769 added with length 5431
GSM11779 added with length 5431
GSM11766 added with length 5431
GSM11775 added with length 5431
GSM11778 added with length 5431
GSM11759 added with length 5431
GSM11774 added with length 5431
GSM11773 added with length 5431
GSM11771 added with length 5431
GSM11758 added with length 5431
GSM11776 added with length 5431
GSM11768 added with length 5431
GSM11770 added with length 5431
GSM11763 added with length 5431
GSM11764 added with length 5431


[Parallel(n_jobs=-1)]: Done   2 tasks      | elapsed: 11.1min
[Parallel(n_jobs=-1)]: Done   9 tasks      | elapsed: 11.4min
[Parallel(n_jobs=-1)]: Done  16 tasks      | elapsed: 11.7min
[Parallel(n_jobs=-1)]: Done  25 tasks      | elapsed: 12.2min
[Parallel(n_jobs=-1)]: Done  37 out of  47 | elapsed: 12.8min remaining:  3.5min
[Parallel(n_jobs=-1)]: Done  42 out of  47 | elapsed: 13.0min remaining:  1.5min


GSM771227 added with length 16913
GSM771243 added with length 16913
GSM771179 added with length 16913
GSM771187 added with length 16913
GSM771158 added with length 16913
GSM771214 added with length 16913
GSM771222 added with length 16913
GSM771221 added with length 16913
GSM771209 added with length 16913
GSM771233 added with length 16913
GSM771215 added with length 16913
GSM771246 added with length 16913
GSM771196 added with length 16913
GSM771212 added with length 16913
GSM771173 added with length 16913
GSM771159 added with length 16913
GSM771183 added with length 16913
GSM771162 added with length 16913
GSM771205 added with length 16913
GSM771232 added with length 16913
GSM771200 added with length 16913
GSM771237 added with length 16913
GSM771163 added with length 16913
GSM771225 added with length 16913
GSM771195 added with length 16913
GSM771210 added with length 16913
GSM771203 added with length 16913
GSM771180 added with length 16913
GSM771172 added with length 16913
GSM771201 adde

[Parallel(n_jobs=-1)]: Done  47 out of  47 | elapsed: 13.2min remaining:    0.0s
[Parallel(n_jobs=-1)]: Done  47 out of  47 | elapsed: 13.2min finished


In [18]:
pickle.dump(X1, open('../datasets/uterineX', 'wb'))
pickle.dump(y1, open('../datasets/uteriney', 'wb'))

pickle.dump(X2, open('../datasets/tongueX', 'wb'))
pickle.dump(y2, open('../datasets/tonguey', 'wb'))